# Labour Market Statistics in Canada

In [5]:
import os
import pandas as pd
from datetime import datetime
import re
from dotenv import dotenv_values


censored_dict = dotenv_values(".env")
directory = censored_dict['FILE_DIRECTORY']
filename = "2020-2024_labour_force_stats.csv"
input_df = pd.read_csv(os.path.join(directory, filename))

In [6]:
input_df.head()

,Labour force characteristics,Data type,20-Mar,20-Apr,20-May,20-Jun,20-Jul,20-Aug,20-Sep,20-Oct,...,23-Jun,23-Jul,23-Aug,23-Sep,23-Oct,23-Nov,23-Dec,24-Jan,24-Feb,24-Mar
0,Population 6 7,"Seasonally adjusted (x 1,000)","24,530.10","24,528.70","24,526.20","24,531.20","24,536.10","24,538.80","24,542.70","24,546.80",...,"25,189.70","25,249.00","25,323.60","25,384.50","25,448.60","25,506.10","25,562.60","25,662.60","25,727.90","25,800.20"
1,NaN,Trend-cycle 8,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,Labour force 9,"Seasonally adjusted (x 1,000)","18,812.00","17,791.80","18,257.20","18,971.60","19,154.80","19,266.90","19,399.40","19,457.30",...,"20,257.70","20,270.30","20,282.10","20,344.80","20,385.30","20,416.20","20,435.80","20,461.80","20,553.60","20,626.30"
3,NaN,Trend-cycle 8,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Employment 10,"Seasonally adjusted (x 1,000)","17,227.00","15,305.60","15,599.10","16,603.40","16,993.20","17,233.50","17,673.10","17,744.10",...,"19,135.60","19,131.00","19,153.00","19,203.40","19,216.80","19,234.30","19,256.20","19,264.90","19,332.60","19,336.70"


In [7]:
# filter out unused rows
input_df = input_df[input_df['Labour force characteristics'].notna()]

In [8]:
input_df.head()

,Labour force characteristics,Data type,20-Mar,20-Apr,20-May,20-Jun,20-Jul,20-Aug,20-Sep,20-Oct,...,23-Jun,23-Jul,23-Aug,23-Sep,23-Oct,23-Nov,23-Dec,24-Jan,24-Feb,24-Mar
0,Population 6 7,"Seasonally adjusted (x 1,000)","24,530.10","24,528.70","24,526.20","24,531.20","24,536.10","24,538.80","24,542.70","24,546.80",...,"25,189.70","25,249.00","25,323.60","25,384.50","25,448.60","25,506.10","25,562.60","25,662.60","25,727.90","25,800.20"
2,Labour force 9,"Seasonally adjusted (x 1,000)","18,812.00","17,791.80","18,257.20","18,971.60","19,154.80","19,266.90","19,399.40","19,457.30",...,"20,257.70","20,270.30","20,282.10","20,344.80","20,385.30","20,416.20","20,435.80","20,461.80","20,553.60","20,626.30"
4,Employment 10,"Seasonally adjusted (x 1,000)","17,227.00","15,305.60","15,599.10","16,603.40","16,993.20","17,233.50","17,673.10","17,744.10",...,"19,135.60","19,131.00","19,153.00","19,203.40","19,216.80","19,234.30","19,256.20","19,264.90","19,332.60","19,336.70"
6,Full-time employment 11,"Seasonally adjusted (x 1,000)","14,553.20","13,138.10","13,358.00","13,898.20","13,961.50","14,108.00","14,529.40","14,534.90",...,"15,947.90","15,920.30","15,948.50","15,964.50","15,989.90","16,028.40","16,027.80","16,029.40","16,099.50","16,110.00"
8,Part-time employment 12,"Seasonally adjusted (x 1,000)","2,673.70","2,167.40","2,241.10","2,705.20","3,031.70","3,125.50","3,143.80","3,209.30",...,"3,187.70","3,210.60","3,204.40","3,239.00","3,226.90","3,205.90","3,228.40","3,235.40","3,233.10","3,226.70"


In [9]:
def change_date_format(col_name):
    '''
        Arg: str
        Return: str
            convert YY-MMM date format to the standard YYYY-MM-DD format
            ex) 20-Feb -> 2020-02-01
            else return original col name
    '''
    if '-' in col_name:
        date_obj = datetime.strptime(col_name, "%y-%b")
        return date_obj.strftime("%Y-%m-%d")
    else:
        return col_name

# this is the dictionary that's going to be used to rename columns
input_df = input_df.drop(columns='Data type')
rename_dict = {col_name:change_date_format(col_name) for col_name in input_df.columns}
not_month_cols = {'Labour force characteristics': 'Date'}
rename_dict.update(not_month_cols)
input_df.rename(columns=rename_dict, inplace=True)

In [10]:
def remove_numbers(text):
    '''
        Arg: str
        Return: str
            remove numeric values from strings
            ex) Employment 12 -> Employment
    '''
    return re.sub(r'\d+', '', text)

input_df['Date'] = [remove_numbers(category) for category in input_df['Date']]

In [11]:
cleaned_df = input_df.transpose()

In [12]:
cleaned_df.head()

,0,2,4,6,8,10,13,15,17
Date,Population,Labour force,Employment,Full-time employment,Part-time employment,Unemployment,Unemployment rate,Participation rate,Employment rate
2020-03-01,"24,530.10","18,812.00","17,227.00","14,553.20","2,673.70","1,585.00",8.4,76.7,70.2
2020-04-01,"24,528.70","17,791.80","15,305.60","13,138.10","2,167.40","2,486.20",14,72.5,62.4
2020-05-01,"24,526.20","18,257.20","15,599.10","13,358.00","2,241.10","2,658.10",14.6,74.4,63.6
2020-06-01,"24,531.20","18,971.60","16,603.40","13,898.20","2,705.20","2,368.20",12.5,77.3,67.7
